In [4]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
https://scvgoe.github.io/

In [ ]:
많은 유저들로부터 모은 취향정보를 바탕으로 예측하는 기술
한 이슈에 대해 A와 B가 같은 의견을 가지면 다른 이슈에 대해서도 비슷한 의견을 보일것이다

In [ ]:
다양한 협업 필터링이 존재.. 그중 memory-based

In [ ]:
유사도를 기반으로 동작
사용자-사용자 : user-based
아이템-아이템 : item-based

In [21]:
df = pd.DataFrame(data = [[5, 4, 4, 3, np.NaN],
                          [1, 0, 1, np.NaN, 4],
                          [4, 4, np.NaN, 5, 3],
                          [np.NaN, 2, 1, 4, 3],
                          [4, np.NaN, 4, 4, 2],
                          [4, 2, 3, np.NaN, 1]],
                  index = ["재석", "명수", "하하", "준하", "세형", "광희"],
                  columns = ["공조", "더킹", "라라랜드", "컨택트", "너의이름은"])

df

,공조,더킹,라라랜드,컨택트,너의이름은
재석,5.0,4.0,4.0,3.0,NaN
명수,1.0,0.0,1.0,NaN,4.0
하하,4.0,4.0,NaN,5.0,3.0
준하,NaN,2.0,1.0,4.0,3.0
세형,4.0,NaN,4.0,4.0,2.0
광희,4.0,2.0,3.0,NaN,1.0


In [ ]:
각 영화를 0~5점으로 평가, nan은 평가를 하지 않은 항목

In [ ]:
user-based
dataframe의 row 하나를 벡터 하나로 생각할 수 있다.
-> row들끼리 비교하여 유사도를 구할 수 있다.
코사인 유사도, 피어슨 유사도를 보통 사용함.
공통적으로 평가한 항목들만 이용함.
ex) 재석, 명수 비교한다하면 -> 공조, 더킹, 라라랜드 3개를 이용해서 유사도 계산

In [ ]:
코사인 유사도를 이용하여 명수와 준하의 유사도를 구한다.
scikit learn에 cosine_similarity가 이미 구현되어있다.

In [22]:
A = np.array(df.loc["명수", df.loc["명수"].notnull() & df.loc["준하"].notnull()]).reshape(1, -1)
B = np.array(df.loc["준하", df.loc["명수"].notnull() & df.loc["준하"].notnull()]).reshape(1, -1)

In [23]:
cosine_similarity(A, B)

array([[0.84266484]])

In [ ]:
이런식으로 모든 사용자에 대한 유사도를 구할 수 있고. 세형의 더킹에 대한 평점을 예측한다고 할때
1. 비슷한 몇명의 점수를 이용하여 예측
2. 전체 대상으로 유사도 기반의 weighted sum으로 구한다.

In [ ]:
item-based
아이템 간의 유사도를 계산..

In [25]:
df_new = df.T
df_new

,재석,명수,하하,준하,세형,광희
공조,5.0,1.0,4.0,NaN,4.0,4.0
더킹,4.0,0.0,4.0,2.0,NaN,2.0
라라랜드,4.0,1.0,NaN,1.0,4.0,3.0
컨택트,3.0,NaN,5.0,4.0,4.0,NaN
너의이름은,NaN,4.0,3.0,3.0,2.0,1.0


In [ ]:
공조와 라라랜드의 유사도를 구한다.

In [27]:
A_new = np.array(df_new.loc["공조", df_new.loc["공조"].notnull() & df_new.loc["라라랜드"].notnull()]).reshape(1, -1)
B_new = np.array(df_new.loc["라라랜드", df_new.loc["공조"].notnull() & df_new.loc["라라랜드"].notnull()]).reshape(1, -1)

In [28]:
cosine_similarity(A_new, B_new)

array([[0.9927901]])

In [ ]:
공조와 라라랜드는 매우 유사도가 높다 -> 공조를 좋아하는 사람은 라라랜드를 좋아할 확률이 높다 or 그반대

In [ ]:
cosine similarity의 단점.
두 벡터가 (5, 5, 5, 5, 5), (1, 1, 1, 1, 1)이라면..평가성향은 매우 반대이지만 유사도는 높게나옴.
개인의 평가성향을 반영하기 힘듬
데이터 수가 적으면 제대로 된 평가를 할 수 없다.

In [ ]:
위 단점을 보완하기 위해 나온 것이 피어슨 유사도. 코사인 유사도에서 사용자의 평균 평가값을 빼준다.

In [ ]:
아마존, 넷플릭스 등 대부분 아이템 기반을 활용.
유저가 아이템을 평가하는 순간 다른 아이템을 추천 해야함 -> 유저의 평가 순간마다 유사도가 업데이트되야함
-> 아이템 기반에서는 미리 아이템간의 유사도를 계산해놓고 활용할 수 있음.
유저수보다 아이템수가 일반적으로 적음